In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os

# Preprering the Dataset by Merging 12 months of sales data into a single CSV file

In [84]:
# Reading the first month the data to have an understanding on what should the data looks like before merging the 12 months together

df = pd.read_csv('Sales_Data/Sales_January_2019.csv')
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


#### Merging the CSV files.

In [15]:
sales_folder = os.path.expanduser('~/Downloads/Sales_Data')

# Get a list of CSV files in the "Sales_Data" folder
csv_files = [file for file in os.listdir(sales_folder) if file.endswith('.csv')]

# Initialize an empty DataFrame to store concatenated data
concatenated_data = pd.DataFrame()

# Loop through each CSV file and concatenate its contents
for file in csv_files:
    file_path = os.path.join(sales_folder, file)
    df = pd.read_csv(file_path)
    concatenated_data = pd.concat([concatenated_data, df], ignore_index=True)

# Save the concatenated data to a new CSV file
output_file = os.path.join(sales_folder, 'concatenated_data.csv')
concatenated_data.to_csv(output_file, index=False)


In [88]:
# Reading the merged CSV files into a DataFrame and droping NaN values.

sales = pd.read_csv('Sales_Data/concatenated_data.csv')
sales.dropna(inplace=True)
sales

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...,...
186845,222905,AAA Batteries (4-pack),1,2.99,06/07/19 19:02,"795 Pine St, Boston, MA 02215"
186846,222906,27in FHD Monitor,1,149.99,06/01/19 19:29,"495 North St, New York City, NY 10001"
186847,222907,USB-C Charging Cable,1,11.95,06/22/19 18:57,"319 Ridge St, San Francisco, CA 94016"
186848,222908,USB-C Charging Cable,1,11.95,06/26/19 18:35,"916 Main St, San Francisco, CA 94016"


# Data Analysis:

####  Question 1: Which month achieved the highest sales, and what was the corresponding earnings? 

To arrive at the answer, I will introduce two new columns. The first column, named "Total Price," will be calculated as the product of the "Quantity Ordered" of each order and its respective "Price Each." The second column, labeled "Month," will involve extracting the month from the "Order Date." This approach streamlines the process of filtering sales data based on the transaction month for later analysis.

In [89]:
# Converting the "Order Date" column to datetime format
sales['Order Date'] = pd.to_datetime(sales['Order Date'], errors='coerce')

# Droping rows with missing "Order Date" values 
sales.dropna(subset=['Order Date'], inplace=True)

# Converting "Price Each" and "Quantity Ordered" to numeric
sales['Price Each'] = pd.to_numeric(sales['Price Each'], errors='coerce')
sales['Quantity Ordered'] = pd.to_numeric(sales['Quantity Ordered'], errors='coerce')

# Creating a new column for total earnings per row
sales['Total Price'] = sales['Price Each'] * sales['Quantity Ordered']

# Extracting the month name
sales['Month'] = sales['Order Date'].dt.strftime('%B')

# Grouping by month and calculate total earnings
monthly_earnings = sales.groupby('Month')['Total Price'].sum()

# Finding the month with the highest earnings
best_month = monthly_earnings.idxmax()
earnings_in_best_month = monthly_earnings.max()

# Printing the results
print(f"The best month of sales was {best_month} with earnings of ${earnings_in_best_month:.2f}")

The best month of sales was December with earnings of $4613443.34


In [90]:
sales

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Total Price,Month
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",1700.00,December
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",600.00,December
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",11.95,December
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",149.99,December
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",11.95,December
...,...,...,...,...,...,...,...,...
186845,222905,AAA Batteries (4-pack),1,2.99,2019-06-07 19:02:00,"795 Pine St, Boston, MA 02215",2.99,June
186846,222906,27in FHD Monitor,1,149.99,2019-06-01 19:29:00,"495 North St, New York City, NY 10001",149.99,June
186847,222907,USB-C Charging Cable,1,11.95,2019-06-22 18:57:00,"319 Ridge St, San Francisco, CA 94016",11.95,June
186848,222908,USB-C Charging Cable,1,11.95,2019-06-26 18:35:00,"916 Main St, San Francisco, CA 94016",11.95,June


#### Displaying monthly total sales and illustrating the results using a histogram.

In [91]:
monthly_earnings = sales.groupby('Month')['Total Price'].sum()

print(monthly_earnings)

Month
April        3390670.24
August       2244467.88
December     4613443.34
February     2202022.42
January      1822256.73
July         2647775.76
June         2577802.26
March        2807100.38
May          3152606.75
November     3199603.20
October      3736726.88
September    2097560.13
Name: Total Price, dtype: float64


In [93]:
# Grouping by month and calculate total price (I also want to display the months in order)

monthly_earnings = sales.groupby('Month')['Total Price'].sum().reset_index()
month_order = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]
monthly_earnings['Month'] = pd.Categorical(monthly_earnings['Month'], categories=month_order, ordered=True)
monthly_earnings = monthly_earnings.sort_values('Month')

fig = px.histogram(monthly_earnings, x='Month', y='Total Price', title='Total Earnings per Month')
fig.update_yaxes(title_text='Total Earnings')
fig.update_traces(marker_color='green')

fig.show()

#### Question 2: Which city recorded the highest number of sales?

To address this, I will create a new column named "City" by extracting information from the existing "Purchase Address" column. This will enable me to calculate earnings on a per-city basis.

In [103]:
# Calculating the  total earnings per row
sales['Total Earnings'] = sales['Price Each'] * sales['Quantity Ordered']

# Extract the city from the "Purchase Address" column
sales['City'] = sales['Purchase Address'].apply(lambda x: x.split(',')[1].strip())

# Group by city and calculate the total earnings
city_earnings = sales.groupby('City')['Total Earnings'].sum().reset_index()

# Find the city with the highest total earnings
highest_earnings_city = city_earnings.loc[city_earnings['Total Earnings'].idxmax()]

print(f"The city with the highest total earnings is {highest_earnings_city['City']} with ${highest_earnings_city['Total Earnings']:.2f} earnings.")

The city with the highest total earnings is San Francisco with $8262203.91 earnings.


In [97]:
sales

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Total Price,Month,Total Earnings,City
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",1700.00,December,1700.00,New York City
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",600.00,December,600.00,New York City
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",11.95,December,11.95,New York City
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",149.99,December,149.99,San Francisco
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",11.95,December,11.95,Atlanta
...,...,...,...,...,...,...,...,...,...,...
186845,222905,AAA Batteries (4-pack),1,2.99,2019-06-07 19:02:00,"795 Pine St, Boston, MA 02215",2.99,June,2.99,Boston
186846,222906,27in FHD Monitor,1,149.99,2019-06-01 19:29:00,"495 North St, New York City, NY 10001",149.99,June,149.99,New York City
186847,222907,USB-C Charging Cable,1,11.95,2019-06-22 18:57:00,"319 Ridge St, San Francisco, CA 94016",11.95,June,11.95,San Francisco
186848,222908,USB-C Charging Cable,1,11.95,2019-06-26 18:35:00,"916 Main St, San Francisco, CA 94016",11.95,June,11.95,San Francisco


#### Visualizing the total earnings per city and presenting the insights through a color-coded scatter plot.

In [123]:
# Group by city and calculate the total earnings
city_earnings = sales.groupby('City')['Total Earnings'].sum().reset_index()

# Sort cities by total earnings in descending order
city_earnings = city_earnings.sort_values('Total Earnings', ascending=False)

# Create a bar plot using plotly.express
fig = px.scatter(city_earnings, x='City', y='Total Earnings', title='Total Earnings per City', color='Total Earnings')

fig.show()

In [125]:
print(city_earnings)

            City  Total Earnings
7  San Francisco      8262203.91
4    Los Angeles      5452570.80
5  New York City      4664317.43
2         Boston      3661642.01
0        Atlanta      2795498.58
3         Dallas      2767975.40
8        Seattle      2747755.48
6       Portland      2320490.61
1         Austin      1819581.75


#### Visualizing city total sales per month and presenting the insights through a color-coded scatter plot.

In [131]:
# Extracting the month from the "Order Date" column
sales['Month'] = sales['Order Date'].dt.strftime('%B')

# Converting the 'Month' column to a categorical variable with custom order
sales['Month'] = pd.Categorical(sales['Month'], categories=month_order, ordered=True)

# Grouping by city and month, and calculate total sales
city_month_earnings = sales.groupby(['City', 'Month'])['Total Earnings'].sum().reset_index()

# Creating a line chart using plotly.express
fig = px.line(city_month_earnings, x='Month', y='Total Earnings', color='City',
              title='City Total Earnings per Month', labels={'Total Earnings': 'Total Sales'})

# Rotating x-axis labels for better readability
fig.update_xaxes(tickangle=45)

# Adding a legend
fig.update_layout(legend=dict(title_text='City'))

fig.show()

#### Question 3: When should we schedule advertisements to optimize the likelihood of customer purchases?

To tackle this question, I will construct a line chart that visualizes the order count throughout the 24-hour day cycle. This approach will aid in pinpointing the hours of peak sales activity and guide our decision-making regarding optimal advertisement timings. 

In [104]:
# Droping rows with missing "Order Date" values
sales.dropna(subset=['Order Date'], inplace=True)

# Extracting the hour of the day
sales['Hour'] = sales['Order Date'].dt.hour

# Grouping by hour and calculate sum of quantity ordered
hourly_quantity = sales.groupby('Hour')['Quantity Ordered'].sum().reset_index()

# Creating a line chart using plotly.express
fig = px.line(hourly_quantity, x='Hour', y='Quantity Ordered', title='Total Quantity Ordered per Hour of the Day',
              labels={'Hour': 'Hour of the Day', 'Quantity Ordered': 'Total Quantity Ordered'})
fig.update_xaxes(range=[0, 24])

fig.show()

My recommendation would be to commence advertising before 11 AM and 7 PM, as indicated by the pronounced peaks in the line chart depicting high sales volumes during these time frames.

#### Question 4: Which products are most frequently sold together?

To address this, I will utilize two columns: "Order ID" to identify items sold within the same transaction and the "Product" column to identify products that are sold together.

In [105]:
# Droping rows with missing Order ID or Product values
sales.dropna(subset=['Order ID', 'Product'], inplace=True)

# Grouping products by Order ID and aggregate the products as lists
product_groups = sales.groupby('Order ID')['Product'].apply(list)

# Initializing a dictionary to store combinations and their frequencies
product_combinations = {}

# Iterating through each group and find combinations
for products in product_groups:
    if len(products) > 1:
        for i, product1 in enumerate(products):
            for j, product2 in enumerate(products):
                if i != j:
                    combination = (product1, product2)
                    if combination in product_combinations:
                        product_combinations[combination] += 1
                    else:
                        product_combinations[combination] = 1

# Sorting the combinations by frequency in descending order
sorted_combinations = sorted(product_combinations.items(), key=lambda x: x[1], reverse=True)

# Printing the most frequently sold together products and their frequencies
for combination, frequency in sorted_combinations[:10]:
    print(f"Products: {combination[0]} and {combination[1]}, Frequency: {frequency}")

Products: iPhone and Lightning Charging Cable, Frequency: 1015
Products: Lightning Charging Cable and iPhone, Frequency: 1015
Products: Google Phone and USB-C Charging Cable, Frequency: 999
Products: USB-C Charging Cable and Google Phone, Frequency: 999
Products: iPhone and Wired Headphones, Frequency: 462
Products: Wired Headphones and iPhone, Frequency: 462
Products: Google Phone and Wired Headphones, Frequency: 423
Products: Wired Headphones and Google Phone, Frequency: 423
Products: iPhone and Apple Airpods Headphones, Frequency: 373
Products: Apple Airpods Headphones and iPhone, Frequency: 373


#### Question 5: Which product achieved the highest sales volume?


For this question, my objective is to identify the product with the highest sales using a histogram. Additionally, I will integrate the prices of these items onto the same graph using a line chart. To achieve this, I will leverage the information from both the "Product" and "Price Each" columns.

In [132]:
# Grouping by product and calculating the total quantity ordered and average price
product_info = sales.groupby('Product').agg({'Quantity Ordered': 'sum', 'Price Each': 'mean'}).reset_index()

# Sorting products by total quantity ordered in descending order
product_info = product_info.sort_values('Quantity Ordered', ascending=False)

# Creating subplots with shared x-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Adding a bar trace for quantity ordered
bar_trace = go.Bar(x=product_info['Product'], y=product_info['Quantity Ordered'], name='Quantity Ordered')
fig.add_trace(bar_trace)

# Adding a line trace for average price
line_trace = go.Scatter(x=product_info['Product'], y=product_info['Price Each'], name='Average Item Price', yaxis='y2')
fig.add_trace(line_trace)

# Seting x-axis and y-axes labels
fig.update_xaxes(tickangle=45)

fig.update_yaxes(title_text='Total Quantity Ordered', secondary_y=False)
fig.update_yaxes(title_text='Average Item Price', secondary_y=True)

# Updating the  layout title
fig.update_layout(title='Total Quantity Ordered and Average Price per Product')

fig.show()